In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 68 bytes


##Loading Dataset


In [2]:
! pip install kaggle

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle competitions download multilingualabusivecomment

 24% 5.00M/21.2M [00:00<00:02, 7.01MB/s]
100% 21.2M/21.2M [00:00<00:00, 25.7MB/s]
 66% 33.0M/50.0M [00:01<00:00, 20.8MB/s]
100% 50.0M/50.0M [00:01<00:00, 41.3MB/s]


In [7]:
! mkdir new

In [8]:
! unzip ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Test_NoLabel.csv.zip -d new

Archive:  ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Test_NoLabel.csv.zip
  inflating: new/ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Test_NoLabel.csv  


In [9]:
! unzip ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Train.csv.zip

Archive:  ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Train.csv.zip
  inflating: ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Train.csv  


In [10]:
import pandas as pd
import re

def read_csv(path):
    file = open(path, "r").read()
    ix = []
    ctx = []

    for row in file.split("\n"):
        l = re.sub(',(?!(?=[^"]*"[^"]*(?:"[^"]*"[^"]*)*$))', "\t", row)
        try:
            lk = l.split("\t")
            if len(lk)>2 and len(lk[0])<6:
                p,q= lk[0], lk[1]
                ix.append(p)
                ctx.append(q)
            else:
                lk=row.replace('"', " ")
                lk=lk.split(",")
                p,q = lk[0], lk[1]
                ix.append(p)
                ctx.append(q)
        except Exception as e:
            print("Exception occurred!.", e)
            print(f"Length of ids obtained: {len(ix)}, and text: {len(ctx)}")

    df = pd.DataFrame()
    df["CommentId"]=ix[1:]
    df["commentText"]=ctx[1:]
    df["commentText"]=ctx[1:]
    df = df.astype(dtype={"CommentId":int, "commentText":str})
    return df

test=read_csv('/content/new/ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Test_NoLabel.csv')

Exception occurred!. list index out of range
Length of ids obtained: 309530, and text: 309530
Exception occurred!. list index out of range
Length of ids obtained: 430869, and text: 430869
Exception occurred!. list index out of range
Length of ids obtained: 641036, and text: 641036


In [11]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
!pip install indic-nlp-library

     |████████████████████████████████| 40 kB 4.0 MB/s 
     |████████████████████████████████| 2.8 MB 34.5 MB/s 


In [13]:
df = pd.read_csv('/content/ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Train.csv',error_bad_lines=False)
print(df.head())

b'Skipping line 446: expected 10 fields, saw 13\nSkipping line 614: expected 10 fields, saw 11\nSkipping line 2576: expected 10 fields, saw 13\nSkipping line 2762: expected 10 fields, saw 18\nSkipping line 3384: expected 10 fields, saw 11\nSkipping line 3831: expected 10 fields, saw 11\nSkipping line 3892: expected 10 fields, saw 12\nSkipping line 4037: expected 10 fields, saw 11\nSkipping line 4042: expected 10 fields, saw 12\nSkipping line 4739: expected 10 fields, saw 16\nSkipping line 4846: expected 10 fields, saw 11\nSkipping line 5306: expected 10 fields, saw 11\nSkipping line 6271: expected 10 fields, saw 16\nSkipping line 7853: expected 10 fields, saw 11\nSkipping line 8970: expected 10 fields, saw 11\nSkipping line 9937: expected 10 fields, saw 16\nSkipping line 10511: expected 10 fields, saw 13\nSkipping line 11199: expected 10 fields, saw 11\nSkipping line 11455: expected 10 fields, saw 11\nSkipping line 11711: expected 10 fields, saw 11\nSkipping line 11785: expected 10 fie

   CommentId  ... label
0          0  ...   1.0
1          1  ...   0.0
2          2  ...   1.0
3          3  ...   0.0
4          4  ...   1.0

[5 rows x 10 columns]


In [14]:
train=df

In [15]:
train.head()

,CommentId,commentText,language,user_index,post_index,report_count_comment,report_count_post,like_count_comment,like_count_post,label
0,0,randi ka deewana,Assamese,2869,175913,0,1,0,65,1.0
1,1,👌👌🙏🥰🥰🥰☝️☝️,Rajasthani,601,960582,0,0,1,94,0.0
2,2,apni pant utar kar apni chut bhi to bata do ma...,Urdu,1205,17327,0,1,0,731,1.0
3,3,krishnatripathi4578 jesi ghar ki sabhyata vais...,Rajasthani,889,352919,0,1,1,285,0.0
4,4,is randi ko dusra kaam nahi hai kuttiya kahin ki,Urdu,372,261223,0,0,0,453,1.0


In [16]:
train['label'].value_counts(normalize=True)

0.0    0.765797
1.0    0.234203
Name: label, dtype: float64

In [17]:
a=['Hindi','English', 'Malayalam','Telugu','Tamil','Punjabi','Kannada','Bengali','Odia', 'Bhojpuri','Marathi','Gujarati','Haryanvi','Rajasthani','Assamese','Urdu']

In [18]:
train = train[train['language'].isin(a)]

In [19]:
train['language'].value_counts(normalize=True)

Hindi         0.332548
English       0.116975
Malayalam     0.108512
Telugu        0.098174
Tamil         0.085325
Punjabi       0.068695
Kannada       0.054439
Bengali       0.031354
Odia          0.025383
Bhojpuri      0.025312
Marathi       0.020679
Gujarati      0.015779
Haryanvi      0.008797
Rajasthani    0.003478
Assamese      0.002403
Urdu          0.002148
Name: language, dtype: float64

In [20]:
lens = train.commentText.str.len()
lens.mean(), lens.std(), lens.max()

(35.786119467183, 54.85568206494333, 22884.0)

In [21]:
label_cols = ['label']

In [22]:
len(train),len(test)

(1483287, 641035)

##Dataset Cleaning


In [23]:
#from teams.models import *
from bs4 import BeautifulSoup
#from django.conf import settings
import requests, os, string
import nltk, string, re, spacy,unicodedata, random
import warnings
warnings.filterwarnings('ignore')

In [24]:
def text_cleaning(row):
  # to remove HTML tags
  text = BeautifulSoup(row, 'html.parser').get_text()
  d = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE) #This line is for removing url
  post = d.replace('\n', '')
  post = post.replace('—', ' ')
  # to remove accented characters
  new_text = unicodedata.normalize('NFKD', post).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  # to remove special characters and numbers
  # define the pattern to keep
  pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
  text = re.sub(pat, '', new_text)
  text = re.sub("\d+", "", text)
  #to remove punctuation
  text = ''.join([c for c in text if c not in string.punctuation])
  # to remove special characters
  pattern = r'^\s*|\s\s*'
  text = re.sub(pattern, ' ', text).strip()
  # convert into lower case
  #text = text.lower() 
  # Stopword Removing
  #tokenizer = ToktokTokenizer()
  # convert sentence into token of words
 # tokens = tokenizer.tokenize(text)
  #tokens = [token.strip() for token in tokens]
  # check in lowercase 
  #t = [token for token in tokens if token.lower() not in stopword_list]
  #text = ' '.join(t) 
  # Stemming
  #t = [stemming.stem(token) for token in tokens ]
  text = ''.join(text) 

  return text

In [25]:
train2=train.copy()

In [26]:
train2 = train2.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

In [27]:
train2['commentCleaned'] = train2['commentText'].apply(text_cleaning)

In [28]:
train2["commentCleaned"].replace("","neutral",inplace=True)

In [29]:
train2['commentCleaned'].isna().sum()

0

In [30]:
train2['commentCleaned'].fillna("neutral", inplace=True)


In [31]:
x=train2["commentCleaned"]
y=train2['label']

In [32]:
x

0                                           randi ka deewana
1                                                    neutral
2          apni pant utar kar apni chut bhi to bata do ma...
3          krishnatripathi jesi ghar ki sabhyata vaise sa...
4           is randi ko dusra kaam nahi hai kuttiya kahin ki
                                 ...                        
1483501                                                pavam
1483502                              enik ind post ittittila
1483503                                    nirmelp parayilla
1483504                                                ettan
1483505                                              neutral
Name: commentCleaned, Length: 1483287, dtype: object

In [33]:
y

0          1.0
1          0.0
2          1.0
3          0.0
4          1.0
          ... 
1483501    0.0
1483502    0.0
1483503    0.0
1483504    0.0
1483505    0.0
Name: label, Length: 1483287, dtype: object

##Train Test Split

In [34]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,shuffle=True,random_state=30)

##Tokenizing and Vectorizing


In [ ]:
import re, string
from indicnlp.tokenize import indic_tokenize
def tokenize(s): return indic_tokenize.trivial_tokenize(s)

In [ ]:
n = train2.shape[0]

vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
                      strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1)


trn_term_doc = vec.fit_transform(x_train)
test_term_doc = vec.transform(x_test)

In [ ]:
trn_term_doc, test_term_doc

(<80000x290580 sparse matrix of type '<class 'numpy.float64'>'
 	with 929208 stored elements in Compressed Sparse Row format>,
 <20000x290580 sparse matrix of type '<class 'numpy.float64'>'
 	with 174846 stored elements in Compressed Sparse Row format>)

In [ ]:
x = trn_term_doc
test_x = test_term_doc

##ML Models

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
import seaborn as sns

##K Neighbors


In [ ]:
model_KN= KNeighborsClassifier(metric= 'euclidean', n_neighbors=19 , weights= 'distance') 
model_KN.fit(x,y_train)
y_predict_KN = model_KN.predict(test_x)
#a=(model_KN.score(test_x,y_test))
#accuracy['BOW'].append(a)
print(metrics.confusion_matrix(y_test, y_predict_KN))
print(metrics.classification_report(y_test, y_predict_KN))
print(metrics.accuracy_score(y_test, y_predict_KN))

[[138523   5359]
 [ 25258  30860]]
              precision    recall  f1-score   support

         0.0       0.85      0.96      0.90    143882
         1.0       0.85      0.55      0.67     56118

    accuracy                           0.85    200000
   macro avg       0.85      0.76      0.78    200000
weighted avg       0.85      0.85      0.84    200000

0.846915
